---
<h1 align=center><strong><em>Template pout le machine learning</em></strong></h1>

---

# 1) Mise en place de l'environnement

## a) Importer toutes les librairies nécessaire

In [1]:
# pour le Dataframe
import pandas as pd 
import numpy as np

# pour les graphiques
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk") 
sns.set_style("darkgrid")

# pour la transformation de données manquante
from sklearn.impute import SimpleImputer

# pour la transformation de données catégorielles
from sklearn.preprocessing import LabelEncoder

# pour splitter le jeu de données en un jeu d'entrainement et de test
from sklearn.model_selection import train_test_split

# pour la mise à l'echelle
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

# pour le choix du tri de variables
from sklearn.feature_selection import VarianceThreshold # permet de pas prendre en compte les variables qui varient que très peu
from sklearn.feature_selection import SelectKBest # permet de garder les variables qui ont le plus d'influence sur la sortie
from sklearn.feature_selection import SelectFromModel # à utiliser avec un estimateur... à approfondir !!!

# pour le choix du model... ça dépend du type et des quantités de données que nous disposons...voir graphique ci-dessous
# exemple :
from sklearn.neighbors import KNeighborsClassifier

# pour la cross validation:
    # pour le tri du jeu d'entrainement
from sklearn.model_selection import KFold # partage le set en n groupes de manière aléatoire 
from sklearn.model_selection import LeaveOneOut # le jeu va être validé sur un seul individu => temps de calculs très long
from sklearn.model_selection import ShuffleSplit # idem KFold sauf que le jeu va être mélanger aprés chaque validation
from sklearn.model_selection import StratifiedKFold # permet de faire des groupes homogènes contenent chacun la même proportion de chaque variables
from sklearn.model_selection import GroupKFold # si une variable est catégorielle => avoir autant de chaque catégorie exemple autant de 1er classe que de 2nd classe
    # pour réaliser le tri sur le jeu d'entrainement et des variables simplement
from sklearn.model_selection import GridSearchCV 

# pour obtenir le score d'un model
from sklearn.model_selection import cross_val_score

# pour la vaildation curve
from sklearn.model_selection import validation_curve # permet de visualiser les résultats avec les différents hyperparamètres

# pour la learning curve
from sklearn.model_selection import learning_curve # permet de savoir s'il y a assez de données ou si plus de données donneraient une meilleure performance

# pour la sauvegarde des meilleurs paramètres
import pickle
import joblib # à approfondir ... différence entre pickle et joblib

# pour un obtenir le temps d'un calcul
from time import time


<center><img src="tableau.png" width="1000" height="700"></center>

## b) Chargement du dataset

In [ ]:
data = pd.read_csv('mon_fichier.csv')
data.head()

# 2) Nettoyage

In [ ]:
data.info() # permet d'avoir une bonne visualisation du dataset, donne les infos suivantes :
            # Nombre de lignes
            # Nom des variables
            # Le type des variables (object, int ,float...)
            # S'il y a des valeurs manquantes

In [ ]:
data.dropna()   # Supprime toutes les lignes où il y a un NaN.
                # Peu conseillé car ça peu supprimer énormémément de données.

In [ ]:
data[data.duplicated()] # permet de voir les lignes exactement identique
data.drop_duplicates()  # permet de garder une seule ligne 

In [ ]:
# Pour traiter une valeur manquante il vaut mieux utiliser SimpleImputer
# qui permet d'affecter une valeur à un NaN suivant une stratégie :
data["categorielle"]= SimpleImputer(strategy='most_frequent', missing_values=np.nan).fit_transform(data[["categorielle"]])
data["continue"] = SimpleImputer(strategy='mean', missing_values=np.nan).fit_transform(data[["continue"]]) # median aurait pu être une statégie ici aussi

In [ ]:
data.isnull().sum() # permet de verifier qu'il n'y a plus de valeur NaN

In [ ]:
data.drop(["inutile_1", "inutile_2"], axis=1) # Supprime des variables inutiles exemple le numéro du ticket dans le dataset du Titanic

In [ ]:
data.info() # pour verifier que tout est en ordre avant l'EDA

# 3) EDA


- L'analyse exploratoire de données (EDA) permet de tirer des conclusions sur le rapport entre les variables et la sortie.
- Elle nous permet aussi d'apprécier la pertinance des résultats suivant la distribution des données (un dataset avec 95% d'hommes ne nous permettra pas d'avoir de conclusions pertinantes sur le rôle du sexe)
- On peut également voir la correlation entre deux variables. S'il y a une correlation proche de 1 entre deux variables cela nous permet d'en supprimer une afin de gagner du temps lors de la construction de notre modèle de machine learning.


## a) Visualisation numérique

In [ ]:
data.describe() # permet d'avoir une vue d'ensemble de chaque variable si elles sont numérique :
                # minimum , maximum, moyenne, écart type...

data["categorielle"].value_counts() # permet de voir les différentes catégories ainsi que leur répartition

## b) Visualisation graphique

In [ ]:
# Matplotlib permet plus de détails et de complexités
# Seaborn utilise matplotlib mais il permet de faire un graphique complexe en une seule ligne

In [ ]:
# Avec matplotlib

plt.figure(figsize=(20,5)) # ouverture d'un nouveau graphique, figsize permet de lui donner une taille. voir doc pour d'autres attributs
plt.scatter(x1, y1, c = 'red') # Crée une graphique avec des points de couleur rouge. voir doc pour autre type de graphique(box,hist...)
plt.scatter(x1,y2, c = 'blue') # il est possible de tracer plusieur courbe sur le même graphique
plt.ylabel('label')
plt.title('Titre_du_graphique')
plt.legend("Pour la légende")
plt.show() # affichage du grapgique

In [ ]:
# Avec Seaborn et plusieurs graphique sur un seul affichage :

fig, ax = plt.subplots(2,2,figsize=(20,15)) # 4 graphique (2 lignes, 2 colonnes) taille totale 20,15 en inch

sns.scatterplot(data=data,x="petal.length",y="petal.width",hue="variety",ax=ax[0,0])    # Lechoix du type de graphique dépend du type de variable
ax[0,0].set_title("Rapport entre largeur et longeur d'une petale")                      # voir doc car beaucoup de possibilités
                                                                                        # boxplot, violinplot, histplot, countplot....
sns.scatterplot(data=data,x="petal.length",y="sepal.length",hue="variety",ax=ax[0,1])
ax[0,1].set_title("Rapport entre largeur d'une pétale et d'un sépal")

sns.scatterplot(data=data,x="petal.width",y="sepal.length",hue="variety",ax=ax[1,0])
ax[1,0].set_title("Rapport entre la largeur d'une pétale et la longueur d'un sépal ")

sns.scatterplot(data=data,x="petal.length",y="sepal.width",hue="variety",ax=ax[1,1])
ax[1,1].set_title("Rapport entre la longeur d'une pétale et la largeur d'un sépal")

plt.show()

In [ ]:
# correlation entre les variables. Tableau de correlations avec les valeurs
sns.heatmap(data.iloc[:,:-1].corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
plt.show()

In [ ]:
# Pour voir les correlations entre des variables (continues de préférence) et une sortie catégorielle
sns.pairplot(data=data.iloc[:,:-1],hue="output")
plt.show()

## c) Conclusion

<h4> Bilan de l'EDA : 
<ul>
<li>Informer sur l'importance ou non d'une variable sur la sortie</li>
<li>Informer sur la distribution des données</li>
<li>Donner les pistes pour notre modèle de machine learning</li>
</ul></h4>

# 4) Transformation de varible catégorielle en variable numérique:

In [ ]:
# Pour une seule variable:
data["variable_transformée"] = LabelEncoder().fit_transform(data["variable_a_tranformer"]) # crée une nouvelle colonne à notre Dataframe

# Pour plusieures variables :
data = LabelEncoder().fit_transform(data.select_dtypes(include=['object'])) # remplace tout sans créer de nouvelles colonnes

# 5) Splitter le dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split("variables", "output", test_size=0.2, random_state=2)

# Le dataset sera splitté en 4 un jeu d'entrainement et jeu de test chacun 
# splitté avec d'un côté les variables et de l'autre la sortie
# test_size permet de definir la taille du jeu de test en pourcent
# Random permet d'avoir toujours le même tirage aléatoire

# 6) Recherche du modèle à utiliser

<center><img src="tableau.png" width="1000" height="700"></center>

# 7) Mise à l'echelle

## a) Choix du scaler avec une méthode graphique

In [1]:
scaler_liste= [StandardScaler(),MinMaxScaler(),RobustScaler()]
for i in scaler_liste:
    scaler = i.fit_transform(X_train)
    plt.figure(figsize=(20,5))
    sns.kdeplot(scaler, legend=False)
    plt.title(f"Distribution du jeu d'entrainement avec la transformation {i}")
    plt.show()

SyntaxError: invalid syntax (2731862569.py, line 1)

## b) Avec le score sans hyperparamètre du model

In [ ]:
scaler_liste,score = [StandardScaler(),MinMaxScaler(),RobustScaler()],0
for i in scaler_liste:
    scaler = i.fit_transform(X_train)
    model = KNeighborsClassifier()
    model.fit(scaler,y_train)
    result = model.score(scaler,y_train)
    print(f"Avec le scaler ", str(i), "le score est de ", result)
    if result>score:
        best_scaler=i
        score=result

# 8) Cross validation

## a) Avec choix manuel de la distribution du jeu d'entrainement

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 20),                  # Dictionnaire avec les hyperparamètres à tester
              'metric': ['euclidean', 'manhattan']}

cv=KFold(5, random_state=42, shuffle=True)                      # Choix manuel de la distribution du jeu d'entrainement
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=cv)  # 
grid.fit(X_train, y_train)                                      # Va entrainer notre model avec toutes les combinaisons d'hyperparamètres possible

print("Le meilleur est de ",grid.best_score_,"avec les parametres", grid.best_params_)

## b) Pour tester les différentes distributions et les différents hyperparamètres avec les performances et le temps de calcul

In [ ]:
param_grid = {'n_neighbors': np.arange(1, 20),
              'metric': ['euclidean', 'manhattan']}

model_selection ={"StatifiedKFold": StratifiedKFold(4),
                "Kfold": KFold(5, random_state=42, shuffle=True),
                "LeaveOneOut": LeaveOneOut(),                           # NE SURTOUT PAS FAIRE SUR UN GROS JEU DE DONNEES TEMPS EXTREMEMENT LONG
                "ShuffleSplit": ShuffleSplit(4, test_size=0.2)}
                
for v in model_selection.values():
    debut = time()
    grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=v)
    grid.fit(X_train, y_train)
    fin = time()


    print("Le meilleur score avec ",str(v).split("(",1)[0],"est",grid.best_score_,"avec les parametres",
        grid.best_params_,"calcumlé en",round(fin-debut,2),"secondes.")


# Il faudra refaire la cross validation avec le model selection qui nous interesse pour sauvegarder les best_params_ et le best_model_selection
best_model_selection=KFold(5, random_state=42, shuffle=True)

# 9) Validation curve

In [ ]:
#--------------------------------------------------
#       
#--------------------------------------------------

# 10) Learning curve

In [ ]:
N, train_score, val_score = learning_curve(model, X_train, y_train,train_sizes=np.linspace(0.05, 1.0, 15), cv=best_model_selection)

print(N)
plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()
plt.show()

# Permet de savoir s'il faut plus de données pour que notre modèle soit plus preformant ou si nous disposons d'assez de données

# 11) Score sur le jeu de test

In [ ]:
score_test = cross_val_score(KNeighborsClassifier(grid.best_params_), best_scaler.transform(X_test), y_test, cv=best_model_selection).mean()
print("Sur le jeu de test on obtient un score de ",score_test*100,"% de bonnes prédictions")

# 12) Sauvegarde des meilleurs paramètres

In [ ]:
model_complet={ "scaler": best_scaler,
                "model":,
                "best_param":best_params_,
                "cross_val":best_model_selection}
joblib.dump(model_complet, 'model_iris.joblib') # à approfondir entre pickle et joblib

# 13) Mise en production